<H1>Capstone Project - The Battle of Neighborhoods

<h2>Introduction

My husband has been offered his dream job a thousand miles from home.
While my job provides me the flexibility to work from a remote location, his new job doesn't have this facility.
So we have decided to relocate somewhere near to his work location.

We are Currently staying in Toronto.

Along with the excitement that I have moving to a new city, I am a littile anxious and nervous as well.  
I was born and raised in Toronto and I have never been away from my parents or Family.
All my friends are here and my kid is also studying at a school here. All the things that we need are easily available nearby.

Both NYU and Toronto cities are very diverse and are the financial capitals of their respective countries. 
We would like compare the neighborhoods of the two cities and determine how similar or dissimilar they are before we move there.

**Audience**
1.  Anyone who is moving to a new city they have never been to. It would help them with settling down in the new environment and exploring new life changing options.
2.	When considering business development strategies, it is often helpful to examine downtown or business district relative to those in peer cities or cities that embody similar qualities. This comparison can help you discover business development opportunities that will strengthen economic vitality in your business district.
3.	This Project will help to determine what kinds of retail or service business activities are supported in like places. 
4.	It will also provide real-life examples of districts that have defined themselves with a clear position in their market in terms of goods and services offered and of primary consumer segments served. 


<h2>Objective

We will use Foursquare Data and Machine Learning segmentation and Clustering to study area classification in details.

The aim of this project is to segment areas of New York City and Toronto based on the most common places captured from Foursquare.
Using segmentation and clustering, we hope we can determine:
1. The similarity or dissimilarirty of both cities
2. Classification of area located inside the city whether it is residential, tourism places, or others

<h2> Data

We are going to use data from below Sources.

For **Toronto** data we are going to use web url: "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" and cleanse it.

**Geocoder_File**: Geospatial_Coordinates.csv

The **New York** neighbourhood data is available free on the internet which has been downloaded to a local file to use it for this assignment.

**Filename**: 
nyu_2451_34572-geojson.json

**FourSquare Data**:

Another aspect of this project is the Foursquare data. I beleive that the data as good as provided,
meaning although we are using Foursquare data for segmentation and clustering, the amount and
accuracy of data captured can't 100% determine correct classification in real world.



**let's Import the required Libraries for this Project**

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import requests

from bs4 import BeautifulSoup


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files



print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


<h3>Load and Explore the data

In [ ]:
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url, 'lxml')
#print(soup.prettify)
my_table = soup.find('table',{'class': "wikitable sortable"})
#creating lists for each of the columns
A=[]
B=[]
C=[]

for row in my_table.findAll("tr"):
    cells = row.findAll("td")
    if len(cells)==3: #Only extract table body not heading
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df=pd.DataFrame(A,columns=['PostCode'])
df['Borough']=B
df['Neighbourhood']=C

df=df[df.Borough != 'Not assigned']

In [20]:
df.Neighbourhood[df.Neighbourhood == "Not assigned"] = df.Borough[df.Neighbourhood == "Not assigned"]
def neighbourhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighbourhood'].tolist()))
grp = df.groupby(['PostCode', 'Borough'])
new_df = grp.apply(neighbourhood_list).reset_index(name='Neighbourhood')

In [21]:
df_geo=pd.read_csv('C:\\Users\\HimaniVerma\\Downloads\\Geospatial_Coordinates.csv')
df_geo = df_geo.rename(columns={'Postal Code': 'PostCode'})
df_geo.drop_duplicates(subset='PostCode', keep="last", inplace=True)

#creating Toronto_df by merging the two dataframes
toronto_df = pd.merge(new_df,df_geo,how='inner', on='PostCode')
toronto_df.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood\n, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [24]:
toronto_df.shape

(103, 5)

In [22]:
#NYU data
with open('C:\\Users\\HimaniVerma\\Downloads\\nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']


# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)


for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


In [23]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [25]:
neighborhoods.shape

(306, 4)

<h3>Plotting Maps of Toronto and New York City

In [28]:
#Map_Toronto

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#Locating Toronto on map

address = 'Toronto, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))


C:\Users\HimaniVerma\Anaconda3.2\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [83]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [31]:
#Map New York City

#Use geopy library to get the latitude and longitude values of New York City

address = 'New York City, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

C:\Users\HimaniVerma\Anaconda3.2\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  import sys


The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [84]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

<h2>Methodology

In this project, We will use the basic methodology.

We have plotted maps of New York City and Toronto using latitude and longitude values.

After this we will use the Foursquare API to explore neighborhoods in both cities.

We will use explore function to get the most common venue categories in each neighborhood,
and then use this feature to group the neighborhoods into clusters

K-means clustering algorithm will be used to complete this task. 

Folium is being used to explore neighborhoods of these cities.

<h5>Let's simplify the above Toronto map and segment and cluster only the neighborhoods in East Toronto.

So let's slice the original dataframe and create a new dataframe of the East Toronto data

In [36]:
ET_data = toronto_df[toronto_df['Borough'] == 'East Toronto'].reset_index(drop=True)

address = 'East Toronto, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))



C:\Users\HimaniVerma\Anaconda3.2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of East Toronto are 43.653963, -79.387207.


In [37]:
# create map of East Toronto using latitude and longitude values
map_ET = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ET_data['Latitude'], ET_data['Longitude'], ET_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ET)  
    
map_ET

<h5>let's simplify the above New York map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

In [38]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
address = 'Manhattan, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

C:\Users\HimaniVerma\Anaconda3.2\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


In [109]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

<h3> Using Foursquare API

In [40]:
CLIENT_ID = 'NQJOE1W124KDOATT0TTHPOZWAP0UBXDDPPMGRUNVJSR3FMPD' # your Foursquare ID
CLIENT_SECRET = '3OIKRWWDUQD1ADAEUCVFTI1ZADE0R3IJBCLUM2LDFFSIPYIN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NQJOE1W124KDOATT0TTHPOZWAP0UBXDDPPMGRUNVJSR3FMPD
CLIENT_SECRET:3OIKRWWDUQD1ADAEUCVFTI1ZADE0R3IJBCLUM2LDFFSIPYIN


<h3> Analyze East Toronto

In [41]:
#Exploring Neighbourhood of East Toronto

neighborhood_latitude = ET_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ET_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = ET_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

In [42]:
# type your answer here
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [43]:
results = requests.get(url).json()

In [44]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [45]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [46]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


Explore Neighborhoods in East Toronto

In [47]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [48]:

ET_venues = getNearbyVenues(names=ET_data['Neighbourhood'],
                                   latitudes=ET_data['Latitude'],
                                   longitudes=ET_data['Longitude'])

The Beaches
Riverdale, The Danforth West

India Bazaar, The Beaches West

Studio District

Business reply mail Processing Centre969 Eastern



In [49]:
print(ET_venues.shape)
ET_venues.head()

(119, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West\n",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [50]:
ET_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business reply mail Processing Centre969 Eastern\n,16,16,16,16,16,16
"India Bazaar, The Beaches West\n",18,18,18,18,18,18
"Riverdale, The Danforth West\n",42,42,42,42,42,42
Studio District\n,39,39,39,39,39,39
The Beaches,4,4,4,4,4,4


In [51]:
print('There are {} uniques categories.'.format(len(ET_venues['Venue Category'].unique())))

There are 67 uniques categories.


In [52]:
# one hot encoding
ET_onehot = pd.get_dummies(ET_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ET_onehot['Neighborhood'] = ET_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ET_onehot.columns[-1]] + list(ET_onehot.columns[:-1])
ET_onehot = ET_onehot[fixed_columns]

ET_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Neighborhood,New American Restaurant,Other Great Outdoors,Park,Pet Store,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Riverdale, The Danforth West\n",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [53]:
ET_onehot.shape

(119, 67)

In [54]:
ET_grouped = ET_onehot.groupby('Neighborhood').mean().reset_index()
ET_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Board Shop,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,New American Restaurant,Other Great Outdoors,Park,Pet Store,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business reply mail Processing Centre969 Easte...,0.062500,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00000,0.000000,0.062500,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.00000,0.000000,0.00000,0.000000,0.0625,0.062500,0.000000,0.000000,0.00000,0.00000,0.0625,0.0625,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.06250,0.000000,0.06250,0.000000,0.000000,0.0625,0.0625,0.06250,0.000000,0.000000,0.000000,0.00000
1,"India Bazaar, The Beaches West\n",0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.00000,0.055556,0.055556,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,0.055556,0.055556,0.000000,0.00000,0.00000,0.0000,0.0000,0.000000,0.000000,0.00000,0.055556,0.000000,0.00000,0.055556,0.00000,0.055556,0.055556,0.000000,0.000000,0.000,0.055556,0.000000,0.055556,0.000000,0.000000,0.00,0.055556,0.055556,0.00000,0.055556,0.00000,0.055556,0.000000,0.0000,0.0000,0.00000,0.000000,0.055556,0.055556,0.00000
2,"Riverdale, The Danforth West\n",0.023810,0.023810,0.0000,0.023810,0.000000,0.000000,0.000000,0.047619,0.023810,0.02381,0.000000,0.000000,0.000000,0.02381,0.000000,0.000000,0.000000,0.095238,0.000000,0.0000,0.000000,0.02381,0.000000,0.02381,0.023810,0.0000,0.000000,0.000000,0.000000,0.02381,0.02381,0.0000,0.0000,0.000000,0.238095,0.02381,0.000000,0.000000,0.02381,0.071429,0.02381,0.000000,0.047619,0.023810,0.000000,0.000,0.023810,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.02381,0.023810,0.02381,0.000000,0.000000,0.0000,0.0000,0.02381,0.000000,0.000000,0.000000,0.02381
3,Studio District\n,0.025641,0.051282,0.0000,0.051282,0.025641,0.025641,0.000000,0.025641,0.025641,0.00000,0.000000,0.000000,0.102564,0.00000,0.025641,0.025641,0.025641,0.076923,0.025641,0.0000,0.025641,0.00000,0.025641,0.00000,0.025641,0.0000,0.000000,0.000000,0.025641,0.00000,0.00000,0.0000,0.0000,0.025641,0.000000,0.00000,0.000000,0.025641,0.00000,0.025641,0.00000,0.000000,0.051282,0.025641,0.025641,0.000,0.000000,0.025641,0.000000,0.025641,0.025641,0.00,0.025641,0.000000,0.00000,0.000000,0.00000,0.025641,0.025641,0.0000,0.0000,0.00000,0.025641,0.000000,0.000000,0.00000
4,The Beaches,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,0.000000,0.000000,0.000000,0.00000,0.00000,0.0000,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.00000,0.250000,0.00000,0.000000,0.000000,0.0000,0.0000,0.00000,0.000000,0.000000,0.000000,0.00000


In [55]:
ET_grouped.shape

(5, 67)

In [56]:
num_top_venues = 5

for hood in ET_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ET_grouped[ET_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business reply mail Processing Centre969 Eastern
----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2         Garden Center  0.06
3        Farmers Market  0.06
4  Fast Food Restaurant  0.06


----India Bazaar, The Beaches West
----
                  venue  freq
0                  Park  0.06
1          Burger Joint  0.06
2  Fast Food Restaurant  0.06
3     Fish & Chips Shop  0.06
4                   Gym  0.06


----Riverdale, The Danforth West
----
                venue  freq
0    Greek Restaurant  0.24
1         Coffee Shop  0.10
2      Ice Cream Shop  0.07
3           Bookstore  0.05
4  Italian Restaurant  0.05


----Studio District
----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Beaches----
                  venue  freq
0           Coffee Shop  0.25
1  Other Great Outdoors  0.25
2                  

#### Let's put that into a *pandas* dataframe

In [96]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ET_grouped['Neighborhood']

for ind in np.arange(ET_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ET_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre969 Easte...,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden
1,"India Bazaar, The Beaches West\n",Fast Food Restaurant,Sushi Restaurant,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater,Burger Joint,Brewery
2,"Riverdale, The Danforth West\n",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Health Food Store,Grocery Store,Furniture / Home Store
3,Studio District\n,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Fish Market,Coworking Space,Convenience Store,Gastropub,Comfort Food Restaurant
4,The Beaches,Coffee Shop,Other Great Outdoors,Pub,Trail,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space


<h3>Cluster Neighborhoods Using K means

In [118]:
# set number of clusters
kclusters = 5

ET_grouped_clustering = ET_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ET_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 2, 1, 0])

In [119]:
#rename neighbourhood column

ET_data.rename(columns={'Neighbourhood' : 'Neighborhood'}, inplace=True)

In [120]:
ET_merged = ET_data

# add clustering labels
ET_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ET_merged = ET_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ET_merged.head() # check the last columns!

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Coffee Shop,Other Great Outdoors,Pub,Trail,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space
1,M4K,East Toronto,"Riverdale, The Danforth West\n",43.679557,-79.352188,4,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Health Food Store,Grocery Store,Furniture / Home Store
2,M4L,East Toronto,"India Bazaar, The Beaches West\n",43.668999,-79.315572,2,Fast Food Restaurant,Sushi Restaurant,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater,Burger Joint,Brewery
3,M4M,East Toronto,Studio District\n,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Fish Market,Coworking Space,Convenience Store,Gastropub,Comfort Food Restaurant
4,M7Y,East Toronto,Business reply mail Processing Centre969 Easte...,43.662744,-79.321558,0,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden


In [121]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ET_merged['Latitude'], ET_merged['Longitude'], ET_merged['Neighborhood'], ET_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3> Analyze Manhattan

In [65]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

# Get URL request
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()


From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [66]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Land & Sea Restaurant,Seafood Restaurant,40.877885,-73.905873


In [67]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

24 venues were returned by Foursquare.


Explore Neighborhoods in Manhattan

In [68]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [69]:

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude'])


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [70]:
print(manhattan_venues.shape)
manhattan_venues.head()
manhattan_venues.groupby('Neighborhood').count()

(3309, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [72]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 332 uniques categories.


Analyze Each Neighborhood

In [73]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,Sou

In [74]:
manhattan_onehot.shape

(3309, 333)

In [75]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
           venue  freq
0    Coffee Shop  0.08
1           Park  0.06
2          Hotel  0.05
3  Women's Store  0.03
4            Gym  0.03


----Carnegie Hill----
            venue  freq
0     Pizza Place  0.06
1     Coffee Shop  0.05
2  Cosmetics Shop  0.05
3            Café  0.04
4             Gym  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1   American Restaurant  0.05
2  Gym / Fitness Center  0.05
3    Chinese Restaurant  0.05
4        Cosmetics Shop  0.05


----Chelsea----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2      Ice Cream Shop  0.05
3              Bakery  0.04
4           Nightclub  0.04


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.09
1  Vietnamese Restaurant  0.04
2           Cocktail Bar  0.04
3    American Restaurant  0.04
4     Dim Sum Restaurant  0.04


----Civic Center----
                  venue  freq
0  Gym / Fitness C

Let's put that into a *pandas* dataframe

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Park,Hotel,Wine Shop,Gym,Women's Store,Italian Restaurant,Cupcake Shop,Pizza Place,Plaza
1,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,Bar,Bookstore,French Restaurant,Gym
2,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Bookstore,Beer Bar,Ethiopian Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,Ice Cream Shop,American Restaurant,Nightclub,Bakery,Hotel,Seafood Restaurant,Theater,Art Gallery
4,Chinatown,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,Vietnamese Restaurant,American Restaurant,Bar,Noodle House,Salon / Barbershop,Ice Cream Shop,Hotpot Restaurant
5,Civic Center,Gym / Fitness Center,Bakery,Italian Restaurant,Yoga Studio,Sporting Goods Shop,Park,Gym,Sandwich Place,Coffee Shop,Cocktail Bar
6,Clinton,Theater,Coffee Shop,Gym / Fitness Center,American Restaurant,Italian Restaurant,Wine Shop,Spa,Hotel,Lounge,Mediterranean Restaurant
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Coffee Shop,Thai Restaurant,Steakhouse,Chinese Restaurant,Taco Place,Street Art
8,East Village,Ice Cream Shop,Bar,Wine Bar,Cocktail Bar,Mexican Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Chinese Restaurant,Speakeasy,Japanese Restaurant
9,Financial District,Coffee Shop,Hotel,Food Truck,Wine Shop,Steakhouse,Bar,Gym,Pizza Place,Park,Italian Restaurant


<H3> K means clustering to Cluster Neighborhoods

In [85]:
# set number of clusters
kclusters = 3

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 0, 0, 2, 0, 0])

In [86]:
manhattan_merged = manhattan_data

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,0,Coffee Shop,Discount Store,Yoga Studio,Diner,Steakhouse,Supplement Shop,Shopping Mall,Shoe Store,Tennis Stadium,Seafood Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,Vietnamese Restaurant,American Restaurant,Bar,Noodle House,Salon / Barbershop,Ice Cream Shop,Hotpot Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Café,Mobile Phone Shop,Bakery,Sandwich Place,Pizza Place,Mexican Restaurant,Shoe Store,Donut Shop,Park,Chinese Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Lounge,Mexican Restaurant,Bakery,Pizza Place,Café,Park,Frozen Yogurt Shop,Spanish Restaurant,Chinese Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Sandwich Place,Yoga Studio


In [87]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<H2> Results

#### East Toronto Cluster 1

In [106]:
ET_merged.loc[ET_merged['Cluster Labels'] == 0, ET_merged.columns[[2] + list(range(5, ET_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Business reply mail Processing Centre969 Easte...,0,Light Rail Station,Garden Center,Pizza Place,Auto Workshop,Brewery,Burrito Place,Comic Shop,Farmers Market,Fast Food Restaurant,Garden


#### East Toronto Cluster 2

In [107]:
ET_merged.loc[ET_merged['Cluster Labels'] == 1, ET_merged.columns[[2] + list(range(5, ET_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,1,Coffee Shop,Other Great Outdoors,Pub,Trail,Chinese Restaurant,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space
1,"Riverdale, The Danforth West\n",1,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Diner,Indian Restaurant,Health Food Store,Grocery Store,Furniture / Home Store
3,Studio District\n,1,Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Fish Market,Coworking Space,Convenience Store,Gastropub,Comfort Food Restaurant


#### East Toronto Cluster 3

In [108]:
ET_merged.loc[ET_merged['Cluster Labels'] == 2, ET_merged.columns[[2] + list(range(5, ET_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"India Bazaar, The Beaches West\n",2,Fast Food Restaurant,Sushi Restaurant,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater,Burger Joint,Brewery


#### Manhattan Cluster 1

In [103]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Coffee Shop,Discount Store,Yoga Studio,Diner,Steakhouse,Supplement Shop,Shopping Mall,Shoe Store,Tennis Stadium,Seafood Restaurant
3,Inwood,Lounge,Mexican Restaurant,Bakery,Pizza Place,Café,Park,Frozen Yogurt Shop,Spanish Restaurant,Chinese Restaurant,Deli / Bodega
5,Manhattanville,Deli / Bodega,Italian Restaurant,Seafood Restaurant,Chinese Restaurant,Mexican Restaurant,Indian Restaurant,Climbing Gym,Lounge,Beer Garden,Falafel Restaurant
6,Central Harlem,African Restaurant,Seafood Restaurant,Cosmetics Shop,American Restaurant,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Bookstore,Beer Bar,Ethiopian Restaurant
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,French Restaurant,Cocktail Bar,Hotel,Boutique,Gym / Fitness Center,Coffee Shop
9,Yorkville,Coffee Shop,Gym,Bar,Italian Restaurant,Pizza Place,Sushi Restaurant,Deli / Bodega,Mexican Restaurant,Japanese Restaurant,Thai Restaurant
10,Lenox Hill,Coffee Shop,Sushi Restaurant,Italian Restaurant,Gym / Fitness Center,Pizza Place,Burger Joint,Gym,Sporting Goods Shop,Art Gallery,Thai Restaurant
11,Roosevelt Island,Park,Deli / Bodega,Sandwich Place,Dog Run,Greek Restaurant,Baseball Field,Gym,Outdoors & Recreation,Coffee Shop,Liquor Store
12,Upper West Side,Italian Restaurant,Bar,Coffee Shop,Bakery,Vegetarian / Vegan Restaurant,Burger Joint,Indian Restaurant,Wine Bar,Seafood Restaurant,Thai Restaurant
14,Clinton,Theater,Coffee Shop,Gym / Fitness Center,American Restaurant,Italian Restaurant,Wine Shop,Spa,Hotel,Lounge,Mediterranean Restaurant


#### Manhattan Cluster 2

In [104]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Carnegie Hill,Pizza Place,Coffee Shop,Cosmetics Shop,Café,Yoga Studio,Spa,Bar,Bookstore,French Restaurant,Gym


#### Manhattan Cluster 3

In [105]:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Cocktail Bar,Dim Sum Restaurant,Vietnamese Restaurant,American Restaurant,Bar,Noodle House,Salon / Barbershop,Ice Cream Shop,Hotpot Restaurant
2,Washington Heights,Café,Mobile Phone Shop,Bakery,Sandwich Place,Pizza Place,Mexican Restaurant,Shoe Store,Donut Shop,Park,Chinese Restaurant
4,Hamilton Heights,Mexican Restaurant,Coffee Shop,Café,Pizza Place,Deli / Bodega,Cocktail Bar,Indian Restaurant,Sushi Restaurant,Sandwich Place,Yoga Studio
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Latin American Restaurant,Coffee Shop,Thai Restaurant,Steakhouse,Chinese Restaurant,Taco Place,Street Art
13,Lincoln Square,Theater,Gym / Fitness Center,Italian Restaurant,Concert Hall,Plaza,French Restaurant,Café,Indie Movie Theater,Performing Arts Venue,Park
15,Midtown,Food Truck,Hotel,Steakhouse,Theater,Coffee Shop,Sporting Goods Shop,American Restaurant,Bakery,Bookstore,Park
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Clothing Store,French Restaurant,Seafood Restaurant,Chinese Restaurant,Indian Restaurant,Electronics Store,Café,Bakery
19,East Village,Ice Cream Shop,Bar,Wine Bar,Cocktail Bar,Mexican Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Chinese Restaurant,Speakeasy,Japanese Restaurant
20,Lower East Side,Café,Coffee Shop,Japanese Restaurant,Sandwich Place,Ramen Restaurant,Pizza Place,Latin American Restaurant,Cocktail Bar,Chinese Restaurant,Shoe Store
22,Little Italy,Bakery,Café,Yoga Studio,Cocktail Bar,Sandwich Place,Salon / Barbershop,Ice Cream Shop,Seafood Restaurant,Chinese Restaurant,Bubble Tea Shop


<h2>Discussion

The K-means algorithm is a technique to cluster n objects, on the basis of attributes, into k partitions, k < n.6 Clustering is the classification of objects into different groups or, more precisely, the partitioning of a dataset into clusters (subsets) so that the data in each cluster (ideally) share some common trait—often proximity according to some defined distance measure

Based on cluster for each cities above, we believe that classification for each cluster can be done better with calculation of venues categories (most common) in each cities. Refering to each clsuter, we can't deterimine clearly what represent in each cluster by using Foursquare - Most Common Venue data.

However, for the sake of this project we assumed each cluster as follow:

* Cluster 1: East Toronto: Residental
* Cluster 2: East Toronto: Mix
* Cluster 3: East Toronto: Health and Fitness
* Cluster 1: Manhattan: Sports, Residental, Shopping
* Cluster 2: Manhattan: Health and Fitness
* Cluster 3: Manhattan: Residential


What is lacking at this point is a systematic, quantitative way to identify and distinguish different district and to describe the correlation most common venues as recorded in Foursquare. The reality is however more complex: similar cities might have or might not have similar common venues. A further step in this classification would be to find a method to extract these common venues and integrate the spatial correlations between different of areas or district.

We believe that the classification we propose is an encouraging step towards a quantitative and systematic comparison of the different cities. Further studies are indeed needed in order to relate the data acquired, then observe it to more meaningful and objective results.

<h2>Conclusion

We can conclude that foursquare Api's are useful in capturing data about various cities in the world. Using it, we refer back to our main objectives, which is to determine;

* The similarity or dissimilarirty of both cities
* Classification of area located inside the city whether it is residential, tourism places, or others

In conclusion, both cities East Toronto and Manhattan are the two most popular cities. However, to declare both cities are similar or dissimilar base on common venues visited is quite difficult. 

Both cities is similar in some venues also dissimilar in certain venues. And for classitification based on common venues, again we must have more systematic or quantitative way to identify and declare this. Comparison can be made, but no such method or quantitative data to determine this. We hope in the future, a method to determine it can be establish and explore for references.

For this particular statement we can conclude that it might take little it of initial adjustment but we will live a comfortable life in New York City. Looking forward to relocate.

Thankyou

Himani Verma